In [5]:
import setup
setup.init_django()

In [6]:
from market import services as market_services
from market import tasks as market_tasks
from market.models import Company

In [7]:
import json


In [8]:
ticker = "C:GBPJPY"
name = "GBPJPY"
company, _ = Company.objects.get_or_create(name=name, ticker=ticker, category="forex")
company.id

3

In [9]:
# market_tasks.sync_historical_data(years_ago=1, company_ids=[company.id], use_celery=False, verbose=True)

# use celery / async
# market_tasks.sync_historical_stock_data.delay(years_ago=5, company_ids=[company.id], use_celery=True, verbose=False)

In [10]:
results = market_services.get_stock_indicators(ticker=ticker, days=60)
results

{'score': 1,
 'ticker': 'C:GBPJPY',
 'indicators': {'ma_5': 191.3702,
  'ma_20': 194.9194,
  'current_price': Decimal('190.1500'),
  'conservative_target': 193.4419,
  'aggressive_target': 195.4756,
  'average_price': 195.3335,
  'avg_volume': 162.55555555555554,
  'latest_volume': 7,
  'volume_change_percent': -95.69377990430623,
  'rsi': 49.7323,
  'avg_gain': 0.4098,
  'avg_loss': 0.4142,
  'period': 14,
  'days': 60,
  'upper_band': Decimal('200.2438'),
  'lower_band': Decimal('190.4233'),
  'sma': Decimal('195.3335'),
  'macd': Decimal('1.9582'),
  'signal': Decimal('0.3830'),
  'histogram': Decimal('1.5752'),
  '23.6%': Decimal('196.5938'),
  '38.2%': Decimal('195.3356'),
  '50.0%': Decimal('194.3188'),
  '61.8%': Decimal('193.3019'),
  '78.6%': Decimal('191.8541')}}

In [11]:
from decimal import Decimal

def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # Or str(obj) if you prefer
    raise TypeError("Object of type %s is not JSON serializable" % type(obj).__name__)

In [12]:
# Convert to JSON
results_as_json = json.dumps(results, default=decimal_default)
results_as_json

'{"score": 1, "ticker": "C:GBPJPY", "indicators": {"ma_5": 191.3702, "ma_20": 194.9194, "current_price": 190.15, "conservative_target": 193.4419, "aggressive_target": 195.4756, "average_price": 195.3335, "avg_volume": 162.55555555555554, "latest_volume": 7, "volume_change_percent": -95.69377990430623, "rsi": 49.7323, "avg_gain": 0.4098, "avg_loss": 0.4142, "period": 14, "days": 60, "upper_band": 200.2438, "lower_band": 190.4233, "sma": 195.3335, "macd": 1.9582, "signal": 0.383, "histogram": 1.5752, "23.6%": 196.5938, "38.2%": 195.3356, "50.0%": 194.3188, "61.8%": 193.3019, "78.6%": 191.8541}}'

In [13]:
OLLAMA_API_URL = "http://localhost:11434/api/chat"
import ollama

In [14]:
res = ollama.chat(
    model="llama3.2",
    messages = [
        {
            "role": "system",
            "content": (
                 "You are a forex trading expert with advanced knowledge of financial indicators. Your goal is to provide actionable recommendations using data in the current month and the last month"
                "to achieve a minimum profit of $2 while maintaining a risk of $1 or less per trade. Ensure that your strategy includes a risk-reward ratio of at least 1:2, "
                "clear entry and exit points, stop loss and take profit levels in pips, and the rationale for your decisions. Base all calculations on a $100 trading account and a $10 trade size. "
                "Only suggest trades that align with prudent risk management and account for market volatility. Respond in valid JSON format."

            )
        },
        {
            "role": "user",
            "content": f"Given the following forex data: {results_as_json}, "
                       "provide a 4hrs prediction and recommendation trade."
        }
    ],
    stream=True,
)

for chunk in res:
    print(chunk["message"]["content"], end="", flush=True)

```json
{
  "prediction": {
    "direction": "bearish",
    "reasoning": [
      "Current price ($190.15) is below the average price ($195.3335), indicating a potential reversal.",
      "RSI (49.7323) is in oversold territory, suggesting a potential bounce back."
    ],
    "target_price": 189.25
  },
  "recommendation": {
    "trade_size": "$10",
    "stop_loss_level": -1.0,
    "take_profit_level": 2.0,
    "risk_reward_ratio": 2.0,
    "entry_points": [
      {
        "price": 190.15,
        "method": "Bollinger Bands"
      },
      {
        "price": 191.3702,
        "method": "MA (5)"
      }
    ],
    "exit_points": [
      {
        "price": 189.25,
        "method": "Target Price"
      },
      {
        "price": 190.4233,
        "method": "Lower Band"
      }
    ]
  }
}
```

Explanation:

*   **Prediction Direction:** Based on the data, we predict a bearish trend in the 4-hour GBP/JPY chart.
*   **Reasoning:**
    *   The current price ($190.15) is below the average p